In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 3.8MB/s 


In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV
from google.colab import drive
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, PReLU
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import sklearn.metrics as metrics
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, plot_roc_curve
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
import kerastuner
from kerastuner.tuners import RandomSearch

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = '/content/gdrive/MyDrive/data'

In [ ]:
df_train = pd.read_csv(path + '/train.csv')
df_train.drop(['ID_code'], axis=1, inplace=True)


In [ ]:
df_test = pd.read_csv(path + '/test.csv')
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in range(df_test.shape[1]):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print('Found',len(real_samples_indexes),'real test')
print('Found',len(synthetic_samples_indexes),'fake test')


d = {}
for i in range(200): d['var_'+str(i)] = 'float32'
d['target'] = 'uint8'
d['ID_code'] = 'object'

Found 100000 real test
Found 100000 fake test


In [ ]:
features = df_train.columns
features = features[1:201]

In [ ]:
x = df_train[features]
y = df_train['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
print(sum(y))

20098


In [ ]:
def create_model(units = 10, dropout_rate =0, epochs = 50, activation='relu'):
    # default values
    activation='relu' 
    dropout_rate=0.0 
    optimizer='adam'
    kernel_initializer = 'uniform'
    momemntum=0
    epochs = 50
    neurons= 10
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=200, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(neurons, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(neurons, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizer, 
                  metrics=tf.keras.metrics.AUC())
    return model

# create model
model = KerasClassifier(build_fn=create_model, batch_size=1000, epochs=2) 

########################################################
activation =  ['relu', 'tanh', 'sigmoid', 'linear'] 
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
neurons = [100,200,300,400,500,600]
##############################################################
# grid search epochs, batch size
batch_size = [128,256,512] 
epochs = [100,200,300,400,500]
param_grid = dict(units = neurons, dropout_rate=dropout_rate, epochs = epochs, batch_size=batch_size)
##############################################################
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x,y) 
##############################################################
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
##############################################################

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored

In [ ]:
def nn_model(hp):
  model=Sequential()
  for i in range(hp.Int('num_layers', 1, 5)):
    model.add(Dense(input_dim=200, units=hp.Int('units_' + str(i),
                                                min_value = 128,
                                                max_value = 512,
                                                step = 32),
                    activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
  model.add(Dense(96, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=tf.keras.metrics.AUC())
  return model

In [ ]:
tuner = RandomSearch(
    nn_model,
    objective=kerastuner.Objective("val_auc", direction="max"),
    max_trials=500,
    executions_per_trial=2,
    directory='final_dir',
    project_name='Neural_Network')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_train, y_train,
             epochs=3,
             validation_data=(x_test, y_test))

Trial 407 Complete [00h 02m 44s]
val_auc: 0.8550169169902802

Best val_auc So Far: 0.8578592538833618
Total elapsed time: 18h 21m 32s

Search: Running Trial #408

Hyperparameter    |Value             |Best Value So Far 
num_layers        |2                 |3                 
units_0           |352               |416               
learning_rate     |0.001             |0.01              
units_1           |256               |256               
units_2           |480               |128               
units_3           |480               |480               
units_4           |320               |480               

Epoch 1/3
5000/5000 [==============================] - 22s 4ms/step - loss: 0.3331 - auc: 0.7257 - val_loss: 0.2597 - val_auc: 0.8440
Epoch 2/3
5000/5000 [==============================] - 20s 4ms/step - loss: 0.2509 - auc: 0.8304 - val_loss: 0.2430 - val_auc: 0.8543
Epoch 3/3
4526/5000 [==========================>...] - ETA: 1s - loss: 0.2471 - auc: 0.8385

In [ ]:
tuner.results_summary()

Results summary
Results in final_dir/Neural_Network
Showing 10 best trials
Objective(name='val_auc', direction='max')


In [ ]:
model = Sequential()
model.add(Dense(416, input_dim=200, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(96, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
              metrics=tf.keras.metrics.AUC())
model.fit(x, y,epochs=1000,batch_size=128)

In [ ]:
test_2 = pd.read_csv(path + '/test.csv')

ids = np.arange(test_2.shape[0])
real_ids = list(set(ids) - set(synthetic_samples_indexes))
real_test = test_2.iloc[real_ids]
fake_test = test_2.iloc[synthetic_samples_indexes]
real_test_id = real_test.ID_code

In [ ]:
df_test =real_test.drop(['ID_code'], axis=1)
result=model.predict(df_test)

In [ ]:
sub = pd.DataFrame()
sub['ID_code']=real_test['ID_code']
sub['target']=result
submission = pd.read_csv(path + '/sample_submission.csv')
new_submission = pd.concat([submission.drop(sub.index,axis = 0),sub],axis=0).sort_index()
new_submission.to_csv(path + '/submission.csv', index=False)

In [ ]:
print(result)

[[0.07839483]
 [0.04321983]
 [0.01361248]
 ...
 [0.00229952]
 [0.05113718]
 [0.04040053]]
